<a href="https://colab.research.google.com/github/lorenafc/MGI/blob/main/AuthorSmallCitiesEncodingandGeocoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import libraries
import pandas as pd
import geopandas as gpd
import requests  # To make API requests
import folium
import numpy as np
from folium import Marker

In [ ]:
from geopandas.tools import geocode

In [ ]:
# upload the file from your computer
from google.colab import files
uploaded=files.upload()

Saving additional_authors.xlsx to additional_authors.xlsx


In [ ]:
# Assuming the uploaded file is named 'additional_authors.xlsx'
file_name = 'additional_authors.xlsx'

# Read the Excel file
authors_small_cities = pd.read_excel(file_name, engine='openpyxl')

authors_small_cities_copy = authors_small_cities

In [ ]:
smalldataset_ten_thousand = authors_small_cities_copy.iloc[:10000,:]
print(smalldataset_ten_thousand.shape)

(10000, 10)


#ENCODING FIX (not all of the names were fixed)

In [ ]:
### Function to fix encoding issues #source chatgpt
### THIS FUNCTION FIXED MOST OF THE LOCATIONS, HOWEVER NOT ALL OF THEM.

#HTML ENCODING INCLUDED

import html  # Import the html module for decoding HTML entities

def fix_encoding(text):
    if not isinstance(text, str):
        return text

    # Decode HTML entities first
    text = html.unescape(text)

    # Define possible encodings
    encodings = ['utf-8', 'latin1', 'windows-1252', 'windows-1250', "latin2", "iso-8859-1", "iso-8859-2",'iso-8859-15', 'macroman', 'ascii', 'cp1250', 'cp1251',
        'cp1253', 'cp1254', 'cp1257', 'iso-8859-2', 'iso-8859-3', 'iso-8859-4',
        'iso-8859-5', 'iso-8859-6', 'iso-8859-7',
        'iso-8859-8', 'iso-8859-9', 'koi8-r', 'koi8-u']

    # Try decoding and encoding combinations
    for enc1 in encodings:
        for enc2 in encodings:
            if enc1 != enc2:
                try:
                    # Try decoding and then encoding with different encodings
                    decoded_text = text.encode(enc2, errors='replace').decode(enc1, errors='replace')
                    # Check if decoded_text makes sense
                    if decoded_text.encode(enc1, errors='replace').decode(enc2, errors='replace') == text:
                        return decoded_text
                except (UnicodeEncodeError, UnicodeDecodeError):
                    continue

    # Try double encoding scenarios
    for enc1 in encodings:
        for enc2 in encodings:
            if enc1 != enc2:
                try:
                    first_pass = text.encode(enc1, errors='replace').decode(enc2, errors='replace')
                    decoded_text = first_pass.encode(enc2, errors='replace').decode(enc1, errors='replace')
                    if decoded_text.encode(enc1, errors='replace').decode(enc2, errors='replace') == text:
                        return decoded_text
                except (UnicodeEncodeError, UnicodeDecodeError):
                    continue

    return text



# Apply the encoding fix function to the columns containing city names
smalldataset_ten_thousand ['borncity'] = smalldataset_ten_thousand['born'].apply(fix_encoding)
smalldataset_ten_thousand ['deathcity'] = smalldataset_ten_thousand ['death'].apply(fix_encoding)
smalldataset_ten_thousand['activecity'] = smalldataset_ten_thousand['active'].apply(fix_encoding)


# Display the first few rows to verify the changes
print(smalldataset_ten_thousand[["born",'borncity', "death", 'deathcity']].head(15))







                          born                 borncity                death  \
0                                                                        NaN   
1                                                                    Cieszin   
2                        (Chur                    (Chur                  NaN   
3                     A Capela                 A Capela                  NaN   
4                    A GudiÃ±a                 A Gudiña          Puebla City   
5        Å ariÅ¡skÃ© Sokolovce       Šarišské Sokolovce                  NaN   
6                     Å ibenik                  Šibenik              Šibenik   
7                     Å ibenik                  Šibenik              Šibenik   
8                     Å ibenik                  Šibenik  Cividale del Friuli   
9                  Å id Serbia               Šid Serbia                  NaN   
10                   Å ivetice                 Šivetice           Prochowice   
11                Å kofja Loka          

<ipython-input-6-bc02a16bd7cd>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smalldataset_ten_thousand ['borncity'] = smalldataset_ten_thousand['born'].apply(fix_encoding)
<ipython-input-6-bc02a16bd7cd>:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smalldataset_ten_thousand ['deathcity'] = smalldataset_ten_thousand ['death'].apply(fix_encoding)
<ipython-input-6-bc02a16bd7cd>:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

In [ ]:
# Apply the encoding fix function to the columns containing city names
# smalldataset['borncity'] = smalldataset['born'].apply(fix_encoding)
# smalldataset['deathcity'] = smalldataset['death'].apply(fix_encoding)
# smalldataset['activecity'] = smalldataset['active'].apply(fix_encoding)


#renaming the columns of the copy
smalldataset_ten_thousand_rename = smalldataset_ten_thousand.rename(columns = {'cleanedbirth':'bitrhyear',
                       'cleaneddeath':'deathyear', "finalname":"nameandbirthdeathyear"},
            inplace = True)
smalldataset_ten_thousand_cleaned = smalldataset_ten_thousand.drop(columns=["originalbirth","originaldeath"])

#change the columns order:
smalldataset_ten_thousand_cleaned=smalldataset_ten_thousand_cleaned[['starturl', 'bitrhyear', 'deathyear', 'nameandbirthdeathyear', 'georeferenceurl', 'born', 'borncity', 'death', 'deathcity','active', 'activecity']]


<ipython-input-7-b8a4a75dc11a>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smalldataset_ten_thousand_rename = smalldataset_ten_thousand.rename(columns = {'cleanedbirth':'bitrhyear',


In [ ]:
# export the result as excel file
file_name = 'smalldataset_ten_thousand_author_cleaned_html.xlsx'
smalldataset_ten_thousand_cleaned.to_excel(file_name)

#GEOCODING

In [ ]:
# source: chatGPT with Lucas Koren suggestions

import pandas as pd
from geopy.geocoders import Nominatim
from geopy.exc import GeopyError
import time
import logging

# Initialize logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Initialize Nominatim geocoder with a user agent
geolocator = Nominatim(user_agent="my_Geocoder_application")

# Function to geocode city names with error handling and rate limiting
def geocode_city(city_name):
    try:
        # Rate limiting
        time.sleep(1)
        # Attempt to geocode the city name with a timeout
        location = geolocator.geocode(city_name, timeout=2)
        if location:
            return (city_name, location.latitude, location.longitude)
        else:
            logging.warning(f"Geocoding failed for: {city_name}")
            return (city_name, None, None)
    except GeopyError as e:
        logging.error(f"Geocoding error for {city_name}: {e}")
        return (city_name, None, None)

# Function to geocode a column of city names and map the results back to the DataFrame
def geocode_column(df, column_name, lat_col_name, lon_col_name):
    # Filter out rows where the column is NaN
    df_filtered = df.dropna(subset=[column_name])

    # Extract unique city names from the DataFrame column
    unique_cities = list(set(df_filtered[column_name]))

    # Geocode unique city names
    geocoded_results = [geocode_city(city) for city in unique_cities]

    # Create a dictionary to map city names to their geocoded coordinates
    geocode_dict = {result[0]: (result[1], result[2]) for result in geocoded_results}

    # Map geocoded coordinates back to the original DataFrame
    df[f'{column_name}_coordinates'] = df[column_name].map(geocode_dict)

    # Extract latitude and longitude from coordinates, filling NaN where necessary
    df[[lat_col_name, lon_col_name]] = df[f'{column_name}_coordinates'].apply(pd.Series)

    # Drop the coordinates column
    df.drop(columns=[f'{column_name}_coordinates'], inplace=True)

    return df

# Example DataFrame
small = pd.DataFrame({
    'borncity': ['Šibenik', 'Sergipe', None, 'Šurany'],
    'deathcity': [None, 'Øvre Eiker', 'Ónod', None],
    'activecity': ['Aarau', None, 'Abdera', 'Abensberg']
})

# Geocode activecity, borncity, and deathcity columns with distinct names for latitude and longitude
columns_to_geocode = [

    ('borncity', 'latitude_born', 'longitude_born'),
    ('deathcity', 'latitude_death', 'longitude_death'),
    ('activecity', 'latitude_active', 'longitude_active')
]

for column, lat_col, lon_col in columns_to_geocode:
    small = geocode_column(small, column, lat_col, lon_col)

print(small.head())

# Applying in a larger dataset
for column, lat_col, lon_col in columns_to_geocode:
    smalldataset_ten_thousand_cleaned = geocode_column(smalldataset_ten_thousand_cleaned, column, lat_col, lon_col)

#Display the DataFrame with geocoded coordinates
print(smalldataset_ten_thousand_cleaned.head(15))


  borncity   deathcity activecity  latitude_born  longitude_born  \
0  Šibenik        None      Aarau      43.741175       15.898446   
1  Sergipe  Øvre Eiker       None     -10.674391      -37.377352   
2     None        Ónod     Abdera            NaN             NaN   
3   Šurany        None  Abensberg      48.086106       18.186834   

   latitude_death  longitude_death  latitude_active  longitude_active  
0             NaN              NaN        47.392715          8.044445  
1       59.744171         9.841215              NaN               NaN  
2       48.001132        20.914416        41.030621        -77.560826  
3             NaN              NaN        48.833241         11.857078  


ERROR:root:Geocoding error for  : Non-successful status code 400


                          starturl bitrhyear  deathyear  \
0    http://viaf.org/viaf/81048146      1570     1634.0   
1   http://viaf.org/viaf/180581759       NaN     1775.0   
2   http://viaf.org/viaf/100161519       NaN     1570.0   
3   http://viaf.org/viaf/169350346      1699     1770.0   
4    http://viaf.org/viaf/67541728      1502     1600.0   
5   http://viaf.org/viaf/121452165      1320     1394.0   
6    http://viaf.org/viaf/62383957      1538     1592.0   
7    http://viaf.org/viaf/67541507      1530     1596.0   
8    http://viaf.org/viaf/78669554      1640     1705.0   
9    http://viaf.org/viaf/79445202      1731     1797.0   
10   http://viaf.org/viaf/24946396      1706     1761.0   
11   http://viaf.org/viaf/59506396      1663     1717.0   
12   http://viaf.org/viaf/84185160      1716     1761.0   
13   http://viaf.org/viaf/59517896       NaN     1604.0   
14   http://viaf.org/viaf/29054119      1731     1776.0   

                          nameandbirthdeathyear  \
0   

In [ ]:
# export the result as excel file
file_name = 'smalldataset_ten_thousand_geocoded_born_death_active.xlsx'
smalldataset_ten_thousand_cleaned.to_excel(file_name)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

# Specify the path in Google Drive
file_path = '/content/drive/My Drive/driv_author_active_bor_death_geocoded.xlsx'

# Save the DataFrame to an Excel file in Google Drive
smalldataset_ten_thousand_cleaned.to_excel(file_path)


In [ ]:
### this code is geocoding the NaN of activecity columns, I didint remove it.

# import pandas as pd
# from geopy.geocoders import Nominatim
# from geopy.exc import GeopyError
# import time
# import logging

# # Initialize logging
# logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# # Initialize Nominatim geocoder with a user agent
# geolocator = Nominatim(user_agent="my_Geocoder_application")

# # Function to geocode city names with error handling and rate limiting
# def geocode_city(city_name):
#     try:
#         # Rate limiting
#         time.sleep(1)
#         # Attempt to geocode the city name with a timeout
#         location = geolocator.geocode(city_name, timeout=1)
#         if location:
#             return (city_name, location.latitude, location.longitude)
#         else:
#             logging.warning(f"Geocoding failed for: {city_name}")
#             return (city_name, None, None)
#     except GeopyError as e:
#         logging.error(f"Geocoding error for {city_name}: {e}")
#         return (city_name, None, None)

# # Filter out rows where activecity is NaN
# smalldataset_1000_filtered = smalldataset_1000_cleaned.dropna(subset=['activecity'])

# # Extract unique city names from the DataFrame (activecity)
# unique_cities_active = list(set(smalldataset_1000_filtered['activecity']))

# # Geocode unique city names
# geocoded_results = [geocode_city(city) for city in unique_cities_active]

# # Create a dictionary to map city names to their geocoded coordinates
# geocode_dict = {result[0]: (result[1], result[2]) for result in geocoded_results if result[1] is not None and result[2] is not None}

# # Map geocoded coordinates back to the original DataFrame
# smalldataset_1000_cleaned['coordinates'] = smalldataset_1000_cleaned['activecity'].map(geocode_dict)
# smalldataset_1000_cleaned[['latitude', 'longitude']] = pd.DataFrame(smalldataset_1000_cleaned['coordinates'].tolist(), index=smalldataset_1000_cleaned.index)
# smalldataset_1000_cleaned.drop(columns=['coordinates'], inplace=True)

# # Display the DataFrame with geocoded coordinates
# print(smalldataset_1000_cleaned.head(15))


In [ ]:
# # Map geocoded coordinates back to the original DataFrame
# smalldataset_1000_cleaned['coordinates'] = smalldataset_1000_cleaned['borncity'].map(geocode_dict)
# smalldataset_1000_cleaned[['latitude', 'longitude']] = pd.DataFrame(smalldataset_1000_cleaned['coordinates'].tolist(), index=smalldataset_1000_cleaned.index)
# smalldataset_1000_cleaned.drop(columns=['coordinates'], inplace=True)
# smalldataset_1000_cleaned_born = smalldataset_1000_cleaned
# # Display the DataFrame with geocoded coordinates
# print(smalldataset_1000_cleaned_born.head(15))

In [ ]:
#### USE small_cities to test the geocoding function



# # Function to geocode the city names source: PyGEO course and chatgpt

# from geopy.geocoders import Nominatim
# from geopy.exc import GeopyError


# # Initialize Nominatim geocoder
# geolocator = Nominatim(user_agent="my_Geocoder_application")

# # Function to geocode city names
# def geocode_city(city_name):
#     try:
#         location = geolocator.geocode(city_name, timeout=10)
#         if location:
#             return (location.latitude, location.longitude)
#         else:
#             print(f"Geocoding failed for: {city_name}")
#             return None
#     except GeopyError as e:
#         print(f"Geocoding error for {city_name}: {e}")
#         return None

# # #Example DataFrame
# # data = {'bornlocation': ['New York', 'Los Angeles', 'Nonexistent City', "A Capela", "A Gudiña", "Šarišské Sokolovce", "Šibenik", "Škvorci u Českého Brodu", "Øvre Eiker"]}
# # authors_small_cities_cleaned = pd.DataFrame(data)

# # # Apply the geocoding function to the 'bornlocation' column
# # authors_small_cities_cleaned['bornlocation_geo'] = authors_small_cities_cleaned['bornlocation'].apply(geocode_city)

# # # Display the first few rows to verify the results
# # print(authors_small_cities_cleaned.head(10))

# # Apply the geocoding function to the 'borncity', 'deathcity', and 'activecity' columns
# small_cities['bornlocation_geo'] = small_cities['bornlocation'].apply(geocode_city)